In [7]:
#%%
import os
import re
import sys
import time
import json

import numpy as np
import pandas as pd
from pandas.core.indexes.base import Index

from tqdm import tqdm


#%%
# !pip install chromedriver_autoinstaller
from bs4 import BeautifulSoup
import chromedriver_autoinstaller

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait

from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException

# from html_table_parser import parser_functions as parser

pd.set_option('mode.chained_assignment', None)

In [16]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1920,1080))
display.start()

In [17]:
webdriver.ChromeOptions()

In [18]:
#%%
url = 'https://www.echemportal.org/echemportal/property-search'

option = webdriver.ChromeOptions()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
option.add_argument('window-size=1920,1080')

driver_path = '/home/rt/chromedriver'
driver = webdriver.Chrome(executable_path="/home/rt/chromedriver", chrome_options=chrome_options)
driver.implicitly_wait(3)
driver.get(url)


#@id는 F12에서 id라고 뜨는 위치다. 그리고 /div는 그 뒤에 몇개의 div가 있냐이고, a[2]는 a _ngcontent-c13이 2개인데, 그중에 2번째꺼를 의미
deselect_path = '//*[@id="datasources-panel-1"]/div/div/div/a[2]'
driver.find_element_by_xpath(deselect_path).click()
time.sleep(0.5)

#위에꺼의 2번째에 보면 div가 아니라 echem-search-sources란 창이 하나 더 있다. 찾기 어렵네 ㄷㄷ 그래서 쭉 찾아보면 div들이 있는 것 중 2번째를 선택하고, 그 안에 체크박스가 있고, div
echa_path = '//*[@id="datasources-panel-1"]/div/echem-search-sources/div/div/div/div[2]/echem-checkbox/div'
driver.find_element_by_xpath(echa_path).click()
time.sleep(0.5)

# add Query Block 버튼
query_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[1]/echem-query-builder/div[2]/div/button'
driver.find_element_by_xpath(query_path).click()
time.sleep(0.5)

# Toxicological information 버튼
tox_button_path = '//*[@id="QU.SE.7-toxicological-information-header"]/div/div[1]/button'
driver.find_element_by_xpath(tox_button_path).click()
time.sleep(0.5)

# tox-to-reproduction-header
tox_to_reproduct_path = '//*[@id="QU.SE.7.8-toxicity-to-reproduction-header"]/div/div[2]'
driver.find_element_by_xpath(tox_to_reproduct_path).click()
time.sleep(0.5)

# header 하위 3개의 창
tox_to_repro_path = '//*[@id="QU.SE.7.8-toxicity-to-reproduction"]/div/div/div[1]/div[3]/button'
driver.find_element_by_xpath(tox_to_repro_path).click()
time.sleep(0.5)

/tmp/ipykernel_650/686095874.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path="/home/rt/chromedriver", chrome_options=chrome_options)


In [19]:
#옆에 새로운 Define이란 창이 열림
#맨 위의 Type of information
info_type_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/echem-property-phrase-field[1]/div/div/div/ng-select/div/div'
driver.find_element_by_xpath(info_type_path).click()
time.sleep(0.5)

# Type 중에서 experimental study를 선택
experiment_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[3]'
driver.find_element_by_xpath(experiment_path).click()
time.sleep(0.5)

#Test guideline 선택
tg_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/echem-property-phrase-field[4]/div/div/div/ng-select/div/div'
driver.find_element_by_xpath(tg_path).click()
time.sleep(0.5)

#TG422경로, 이건 바꿔야함
tg422_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[17]'
driver.find_element_by_xpath(tg422_path).click()
time.sleep(0.5)

#이건 저장이니 안건드려도 됨
save_path = '/html/body/echem-root/div/echem-substance-search-page/echem-substance-search-container/echem-substance-search/form/div/ngb-accordion[1]/div[2]/div[2]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/div/button[2]'
driver.find_element_by_xpath(save_path).click()
time.sleep(0.5)

#이것도 검색시작이므로 안건드려도 됨
search_path = '/html/body/echem-root/div/echem-substance-search-page/echem-substance-search-container/echem-substance-search/form/div/div[2]/div/button'
driver.find_element_by_xpath(search_path).click()

In [20]:
#%%
result_ = []

page_num_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-pagination/div/div[2]'
page_num = int(driver.find_element_by_xpath(page_num_path).text.split(' ')[-1])

In [21]:
start = time.time()
for p in range(1, page_num + 1):
#for p in range(1, 4):
    row_num_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-property-search-results/table/tbody/tr'
    row_num = len(driver.find_elements_by_xpath(row_num_path))
    
    row = tqdm(range(1, row_num + 1), file = sys.stdout)
    
    for i in row:
        src_dict = {}
        
        chem_path = '//*[@id="top"]/echem-substance-search-page/echem-property-search-results-container/echem-property-search-results/table/tbody/tr[%d]/td[3]/a'
        property_url = driver.find_element_by_xpath(chem_path % i).get_attribute('href')
        src_dict['link'] = property_url
        
        driver.execute_script('window.open('');')
        driver.switch_to.window(driver.window_handles[1])
        driver.get(property_url)
        time.sleep(0.5)
        
        try:
            accept_path = '/html/body/div[1]/div/div[2]/div[2]/button[1]'
            driver.find_element_by_xpath(accept_path).send_keys(Keys.ENTER)
        except NoSuchElementException:
            pass
        
        try:
            src = driver.page_source
            # soup = BeautifulSoup(src, 'html.parser')
            src_dict['src'] = src
            
            result_.append(src_dict)
        
        except AttributeError:
            pass

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        row.set_postfix({'page': p})
    
    if p < page_num:
        next_page_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-pagination/div/div[2]/a[3]'
        driver.find_element_by_xpath(next_page_path).click()
        time.sleep(1.5)
    
    p += 1


print(time.time() - start)


#%%
json.dump(result_, open('tg422_reproduct_source.json', 'w'))

100%|███████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.18s/it, page=576]
7410.059178113937


## json전처리

In [184]:
import re
import json

import numpy as np
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup


#%%
with open('tg422_reproduct_source.json', 'r') as file:
    df = pd.DataFrame(json.load(file))

#%%
def remove_bracket(string):
    clean_string = re.sub('<.*?>', '', str(string))
    return clean_string

#%%
result_ = []

for i in tqdm(range(len(df))):
    CS = np.nan
    WC = np.nan
    MV = np.nan
    RE = np.nan
    RP = np.nan
    EL = np.nan
    try: 
        soup = BeautifulSoup(df.src[i], 'html.parser')
        chem_dict = {}
        
        # chemical name
        chem_name = soup.find('div', attrs = {'id': 'SubstanceName'}).find_next('h1').text
        chem_dict['Chemical'] = chem_name


        # casrn
        casrn_tmp = soup.find('div', attrs = {'class': 'container'}).find_next('strong').text
        casrn = re.sub('\n|\t', '', casrn_tmp).split( )[-1]
        chem_dict['CasRN'] = casrn

            
        # Clinical signs:
        try:
            admin_tmp = soup.find_all("dl", {"class" : "HorDL"})
            
            for i in range(len(admin_tmp)):
                if "<dt>Clinical signs:" in str(admin_tmp[i]):
                    CS = str(admin_tmp[i]).split('<dt>Clinical signs:</dt><dd>')[1]
                    CS = CS.split('</dd>')[0]
                chem_dict['Clinical signs'] = CS
        except ValueError:
            chem_dict['Clinical signs'] = np.nan

        # Body weight and weight changes:
        try:
            admin_tmp = soup.find_all("dl", {"class" : "HorDL"})
            
            for i in range(len(admin_tmp)):
                if "<dt>Body weight and weight changes" in str(admin_tmp[i]):
                    WC = str(admin_tmp[i]).split('<dt>Body weight and weight changes:</dt><dd>')[1]
                    WC = WC.split('</dd>')[0]
                chem_dict['weight changes'] = WC
        except ValueError:
            chem_dict['weight changes'] = np.nan
            
        # Mortality / viability:
        try:
            admin_tmp = soup.find_all("dl", {"class" : "HorDL"})
            
            for i in range(len(admin_tmp)):
                if "<dt>Mortality / viability" in str(admin_tmp[i]):
                    MV = str(admin_tmp[i]).split('<dt>Mortality / viability:</dt><dd>')[1]
                    MV = MV.split('</dd>')[0]
                chem_dict['Mortality viability'] = MV
        except ValueError:
            chem_dict['Mortality viability'] = np.nan
            
        # Reproductive effects observed2
        try:
            admin_tmp = soup.find_all("dl", {"class" : "HorDL"})
            
            for i in range(len(admin_tmp)):
                if "<dt>Reproductive effects observed" in str(admin_tmp[i]):
                    RE = str(admin_tmp[i]).split('<dt>Reproductive effects observed:</dt><dd>')[1]
                    RE = RE.split('</dd>')[0]
                chem_dict['Reproductive effects observed2'] = RE
        except ValueError:
            chem_dict['Reproductive effects observed2'] = np.nan
            
        # Reproductive performance:
        try:
            admin_tmp = soup.find_all("dl", {"class" : "HorDL"})
            
            for i in range(len(admin_tmp)):
                if "<dt>Reproductive performance" in str(admin_tmp[i]):
                    RP = str(admin_tmp[i]).split('<dt>Reproductive performance:</dt><dd>')[1]
                    RP = RP.split('</dd>')[0]
                chem_dict['Reproductive performance'] = RP
        except ValueError:
            chem_dict['Reproductive performance'] = np.nan
            
        # Basis for effect level:
        try:
            admin_tmp = soup.find_all("dl", {"class" : "HorDL"})
            
            for i in range(len(admin_tmp)):
                if "<dt>Basis for effect level:" in str(admin_tmp[i]):
                    EL = str(admin_tmp[i]).split('<dt>Basis for effect level:</dt><dd>')[1]
                    EL = EL.split('</dd>')[0]
                chem_dict['effect level'] = EL
        except ValueError:
            chem_dict['effect level'] = np.nan
            
            
        for tab in table_list:
            chem_dict_ = chem_dict.copy()
            
            key = [re.sub(':', '', i.text).strip() for i in tab.find_all('dt')]
            value = [i.text.strip() for i in tab.find_all('dd')]
                                    
            if len(key) == len(value) and key[0] != '' and value[0] != 'Key result':
                result_dict = dict(zip(key, value))
                # result_dict = {key[i]: re.sub('<.*?>', '', cell.text).strip() for i, cell in enumerate(tab.find_all('dd'))}
            
            elif len(key) == len(value) and key[0] == '' and value[0] == 'Key result':
                result_dict = dict(zip(key[1:], value[1:]))
            
            elif len(key) != len(value) and key[0] == '' and value[0] == 'Key result':
                key = key[1:]
                value_ = value[1:len(key)] + ['. '.join(value[len(key):])]
                result_dict = dict(zip(key, value_))
            
            chem_dict_.update(result_dict)
            result_.append(chem_dict_)

    except AttributeError:
        pass
    


# %%
result = pd.DataFrame(result_)

100%|███████████████████████████████████████████████████████████████████████████████| 2877/2877 [06:14<00:00,  7.69it/s]


In [183]:
result

Chemical       CasRN  \
0                                 Triphenyl phosphite    101-02-0   
1                                   Methylcyclohexane    108-87-2   
2                1,1,1,3,5,5,5-heptamethyltrisiloxane   1873-88-7   
3                            Trimethoxy(methyl)silane   1185-55-3   
4                   2-(2H-benzotriazol-2-yl)-p-cresol   2440-22-4   
5                                              Iodine   7553-56-2   
6                       Dodecamethylcyclohexasiloxane    540-97-6   
7   [1,3(or 1,4)-phenylenebis(1-methylethylidene)]...  25155-25-3   
8                  4,4'-methylenebis[2-chloroaniline]    101-14-4   
9                    Tris(2-methoxyethoxy)vinylsilane   1067-53-4   
10                             Di-tert-butyl peroxide    110-05-4   
11                           Hexamethylene diacrylate  13048-33-4   
12                                Triphenyl phosphite    101-02-0   
13                                Triphenyl phosphite    101-02-0   
14                                 Trixylyl phosphate  25155-23-1   
15   2,2'-dimethyl-4,4'-methylenebis(cyclohexylamine)   6864-37-5   
16                                     Silver nitrate   7761-88-8   
17                   O,O,O-triphenyl phosphorothioate    597-82-0   
18                   O,O,O-triphenyl phosphorothioate    597-82-0   
19                                            Xylenol   1300-71-6   

   Reproductive effects observed1 Reproductive effects observed2  \
0                             NaN                            NaN   
1                   not specified                  not specified   
2                             NaN                             no   
3                             NaN                             no   
4                             NaN                             no   
5                             NaN                  not specified   
6                             NaN                             no   
7                             NaN                  not specified   
8                   not specified                  not specified   
9                             NaN                            yes   
10                            NaN                  not specified   
11                            NaN                  not specified   
12                            NaN                  not specified   
13                            NaN                  not specified   
14                            NaN                  not specified   
15                            NaN                            yes   
16                            NaN                  not specified   
17                            NaN                             no   
18                            NaN                             no   
19                            NaN                  not specified   

               Reproductive performance Dose descriptor      Effect level  \
0                   no effects observed           NOAEL  25  mg/kg bw/day   
1                   no effects observed           NOAEL  25  mg/kg bw/day   
2                   no effects observed           NOAEL  25  mg/kg bw/day   
3                   no effects observed           NOAEL  25  mg/kg bw/day   
4                   no effects observed           NOAEL  25  mg/kg bw/day   
5                   no effects observed           NOAEL  25  mg/kg bw/day   
6                   no effects observed           NOAEL  25  mg/kg bw/day   
7   effects observed, treatment-related           NOAEL  25  mg/kg bw/day   
8                   no effects observed           NOAEL  25  mg/kg bw/day   
9   effects observed, treatment-related           NOAEL  25  mg/kg bw/day   
10                  no effects observed           NOAEL  25  mg/kg bw/day   
11                  no effects observed           NOAEL  25  mg/kg bw/day   
12                  no effects observed           NOAEL  25  mg/kg bw/day   
13                  no effects observed           NOAEL  25  mg/kg bw/day   
14  effects

In [164]:
df1 = df[0:20]

soup = BeautifulSoup(df.src[15], 'html.parser')
chem_dict = {}
admin_tmp = soup.find_all("dl", {"class" : "HorDL"})

for i in range(len(admin_tmp)):
    if "<dt>Reproductive effects observed" in str(admin_tmp[i]):
        result = str(admin_tmp[i]).split('<dt>Reproductive effects observed:</dt><dd>')[1]
        result = result.split('</dd>')[0]
        chem_dict['Reproductive effects observed'] = result
        print(result)
        #print(admin_tmp[i])

yes


In [152]:
df1 = df[0:20]

soup = BeautifulSoup(df.src[17], 'html.parser')
chem_dict = {}
admin_tmp = soup.find_all("dl", {"class" : "HorDL"})

for i in range(len(admin_tmp)):
    if "<dt>Reproductive performance" in str(admin_tmp[i]):
        result = str(admin_tmp[i]).split('dd>')[1][:-2]
        chem_dict['Reproductive performance'] = result
        print(result)
        #print(admin_tmp[i])

effects observed, non-treatment-related


In [189]:
result['Reproductive performance'].value_counts()

no effects observed                        1873
effects observed, non-treatment-related     450
effects observed, treatment-related         363
not examined                                 94
not specified                                34
Name: Reproductive performance, dtype: int64

In [190]:
result['Reproductive effects observed1'].value_counts()

not specified    115
Name: Reproductive effects observed1, dtype: int64

In [191]:
result['Reproductive effects observed2'].value_counts()

not specified    1166
no               1121
yes               526
Name: Reproductive effects observed2, dtype: int64

In [192]:

#%%
'''
    1. effects observed가 하나라도 있으면 1
    1-1. no effects observed가 있으면 0
    
'''

def find_colon(string):
    return ':' in string


def check_nan(string):
    return string == string
        

# result['Route of administration'].unique()
# result['Route of administration'].value_counts()

# effect 항목의 결과 내용 중 포함이 될 경우
def effective(string):
    if check_nan(string):
        string = string.lower()
        if 'effects observed' in string and 'no effects observed' not in string:
            type_ = 1
            
        elif 'yes' in string:
            type_ = 1
        
        elif 'no' in string or 'not' not in string:
            type_ = 0
            
        else:
            type_ = np.nan
    
    else:
        type_ = np.nan
        
    return type_

#%%
df = result.copy()

# effect
df['Reproductive effects observed1'] = df['Reproductive effects observed1'].map(lambda x: effective(x))
df['Reproductive effects observed2'] = df['Reproductive effects observed2'].map(lambda x: effective(x))
df['Reproductive performance'] = df['Reproductive performance'].map(lambda x: effective(x))

df = df[['CasRN', 'Reproductive effects observed1', 'Reproductive effects observed2', 'Reproductive performance']]

In [193]:
for i in range(len(df)):
    if df['Reproductive effects observed1'][i] == 1.0 or df['Reproductive effects observed2'][i] == 1.0 or df['Reproductive performance'][i] == 1.0:
        target = 1
    else : target = 0
    df.loc[i,'target'] = target

In [195]:
# save df
df = df[['CasRN', 'target']]
df.to_csv('tg422_repro_pre1.csv', header = True, index = False)

# smiles 추가

In [196]:
import cirpy
import openpyxl
import warnings

import pandas as pd
import numpy as np

from tqdm import tqdm

pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings("ignore")

In [198]:
data = pd.read_csv('tg422_repro_pre1.csv')
len(data['CasRN'].unique())

1704

In [199]:
data.head()

CasRN  target
0   101-02-0     0.0
1   108-87-2     0.0
2  1873-88-7     0.0
3  1185-55-3     0.0
4  2440-22-4     0.0

In [200]:
data['target'].unique()
data['target'].isna().sum()

data = data[data['target'].notna()]

casrn_na_idx = data[data['CasRN'] == '-'].index

data = data.drop(casrn_na_idx).reset_index(drop = True)

target_na_idx = data[data['target'] == ''].index

data = data.drop(target_na_idx).reset_index(drop=True)

In [201]:
data = data.drop_duplicates()
len(data)

1711

In [202]:
data1 = data[:600]
data2 = data[600:1200]
data3 = data[1200:]

In [203]:
tqdm.pandas()
data1['SMILES'] = data1.CasRN.progress_apply(lambda x: cirpy.resolve(x, 'smiles'))

100%|█████████████████████████████████████████████████████████████████████████████████| 600/600 [13:18<00:00,  1.33s/it]


In [204]:
tqdm.pandas()
data2['SMILES'] = data2.CasRN.progress_apply(lambda x: cirpy.resolve(x, 'smiles'))

100%|█████████████████████████████████████████████████████████████████████████████████| 600/600 [12:40<00:00,  1.27s/it]


In [205]:
tqdm.pandas()
data3['SMILES'] = data3.CasRN.progress_apply(lambda x: cirpy.resolve(x, 'smiles'))

100%|█████████████████████████████████████████████████████████████████████████████████| 511/511 [10:59<00:00,  1.29s/it]


In [211]:
data = pd.concat([data1, data2, data3])
data = data[['CasRN','SMILES','target']]

In [212]:
data

CasRN                            SMILES  target
0      101-02-0  O(P(Oc1ccccc1)Oc2ccccc2)c3ccccc3     0.0
1      108-87-2                         CC1CCCCC1     0.0
2     1873-88-7   C[Si](O[Si](C)(C)C)O[Si](C)(C)C     0.0
3     1185-55-3                   CO[Si](C)(OC)OC     0.0
4     2440-22-4       Cc1ccc(O)c(c1)n2nc3ccccc3n2     0.0
...         ...                               ...     ...
2342  2867-47-2               CN(C)CCOC(=O)C(C)=C     1.0
2343   112-53-8                     CCCCCCCCCCCCO     0.0
2346   100-42-5                       C=Cc1ccccc1     0.0
2348   592-41-6                           CCCCC=C     1.0
2355    97-88-1                  CCCCOC(=O)C(C)=C     0.0

[1711 rows x 3 columns]

In [213]:
data.SMILES.isna().sum()
data = data[data['CasRN'].notna()].reset_index(drop = True)

In [214]:
data.to_csv('tg422_repro_2smiles.csv', header = True, index = False)

In [ ]:
## 